In [ ]:
model = YOLO("yolo11n.pt")
image = cv.imread('../data/demo.jpg')
results = model(image, classes=[1])
for x,y,w,h in results[0].boxes.xywh:
    x, y, w, h = int(x.item()), int(y.item()), int(w.item()), int(h.item())
    dummy = cv.rectangle(image, (int(x-w/2), int(y-h/2)), (int(x + w/2), int(y + h/2)), (255, 0, 0), 2) 
plt.imshow(dummy)

In [ ]:
my_dict = model.names

values_to_find = ['banana', 'orange']

# Create a dictionary to store the result
keys_for_values = {value: [] for value in values_to_find}

# Loop through the dictionary and find keys for each value
for key, value in my_dict.items():
    if value in values_to_find:
        keys_for_values[value] = key

print(keys_for_values)

In [ ]:
for x,y,w,h in results[0].boxes.xywh:
    x, y, w, h = int(x.item()), int(y.item()), int(w.item()), int(h.item())
    print(x, y, w, h)

In [ ]:
result = model(image,classes=[1], show_boxes=True)

In [ ]:
plt.imshow(results[0].orig_img)